# 3. Ranking Model

- 해당 notebook에선 ```2.Candidates Generate```에서 article 후보군에 대해 고객별로 다음에 가장 구매할 확률이 높은 12개의 article들을 순서를 고려하여 추천하는 모델을 만들 것이다.

<hr>

# Index

## 0. Ranking 모델을 사용하기 위한 Data 전처리
    - Train-Validation split & Candidates load
    - DataFrame 변환 및 Labeling
    - Customer & Article 메타 데이터 추가
    
## 1. LightGBM Ranker 학습
    - Train, Validation 데이터 load
    - LightGBM Ranker training
    
## 2. Ranker를 활용한 추천 및 검증
    - 고객별 추천(with validation data)
    - MAP@12(with validation data)
    - 성능 평가에 대한 고찰
<hr>

## 0. Ranking 모델을 사용하기 위한 Data 전처리
    - Train-Validation split & Candidates load
    - DataFrame 변환 및 Labeling
    - Customer & Article 메타 데이터 추가

In [1]:
import pandas as pd
import numpy as np
import pickle
import joblib
from datetime import datetime, timedelta
from collections import defaultdict
from tqdm import tqdm

### Train-Validation split & Candidates load

In [2]:
# transactions_train.csv load
transactions = pd.read_csv('transactions_train.csv')
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'], format='%Y-%m-%d')

# train, validaion split
first_date, last_date = min(transactions['t_dat']), max(transactions['t_dat'])
last_one_week = last_date + timedelta(weeks=-1, days=1)
train = transactions[transactions['t_dat']<last_one_week].copy()
validation = transactions[transactions['t_dat']>=last_one_week].copy()
train = train[['customer_id', 'article_id']]
validation = validation[['customer_id', 'article_id']]

# exclude customers who did their first purchase in validaion-period 
customer_to_include = np.intersect1d(validation['customer_id'].unique(),train['customer_id'].unique(),\
                                     assume_unique=True)
article_to_include = np.intersect1d(validation['article_id'].unique(), train['article_id'].unique(),\
                                    assume_unique=True)
validation_df = validation[validation['customer_id'].isin(customer_to_include)]
validation_df = validation_df[validation_df['article_id'].isin(article_to_include)]

# candidates load
customer_candidates_dict = np.load('total_candidates.npy', allow_pickle=True)[0]

- ```transactions_train.csv``` 데이터를 불러와서 가장 최근 일주일동안의 구매기록은 validation 데이터로, 나머지는 train 데이터로 분리한다. 이후 validation 기간 동안 첫 구매를 한 고객들, 즉 training 기간엔 구매기록이 없는 고객들을 제외시켜서 validaion 데이터를 구성한다.
- 그리고 고객별로 추천된 article 후보군이 저장된 ```customer_candidates_dict```을 load한다.

### DataFrame 변환 및 Labeling

In [17]:
# all customer id's in training data
customer_id_list = train['customer_id'].unique()

# Turn candidates dictionary into DataFrame
# Split processing because of the limit of I/O speed stem from growing too large variables
# 0 ~ 10,000
temp_id = customer_id_list[0]
df = {'customer_id': [temp_id]*len(customer_candidates_dict[temp_id]),\
      'article_id': customer_candidates_dict[temp_id]}
temp_df = pd.DataFrame(data = df)
temp_candidates = temp_df.copy()
for j in range(1, 10000):
        cus_id = customer_id_list[j]
        df = {'customer_id': [cus_id]*len(customer_candidates_dict[cus_id]),\
              'article_id': customer_candidates_dict[cus_id]}
        temp_candidates = pd.concat([temp_candidates, pd.DataFrame(data = df)]).reset_index(drop=True)
total_candidates_df = temp_candidates.copy()

# 10,000 ~ 1,350,000
for i in range(10000, customer_id_list.shape[0]-10000, 10000):
    temp_id = customer_id_list[i]
    df = {'customer_id': [temp_id]*len(customer_candidates_dict[temp_id]),\
          'article_id': customer_candidates_dict[temp_id]}
    temp_df = pd.DataFrame(data = df)
    temp_candidates = temp_df.copy()
    for j in range(i+1, i+10000):
        cus_id = customer_id_list[j]
        df = {'customer_id': [cus_id]*len(customer_candidates_dict[cus_id]),\
              'article_id': customer_candidates_dict[cus_id]}
        temp_candidates = pd.concat([temp_candidates, pd.DataFrame(data = df)]).reset_index(drop=True)
    total_candidates_df = pd.concat([total_candidates_df, temp_candidates]).reset_index(drop=True)

# 1,350,000 ~ end
temp_id = customer_id_list[1350000]
df = {'customer_id': [temp_id]*len(customer_candidates_dict[temp_id]),\
      'article_id': customer_candidates_dict[temp_id]}
temp_df = pd.DataFrame(data = df)
temp_candidates = temp_df.copy()
for i in range(1350001, customer_id_list.shape[0]):
    temp_id = customer_id_list[i]
    df = {'customer_id': [temp_id]*len(customer_candidates_dict[temp_id]),\
          'article_id': customer_candidates_dict[temp_id]}
    temp_candidates = pd.concat([temp_candidates, pd.DataFrame(data = df)]).reset_index(drop=True)
total_candidates_df = pd.concat([total_candidates_df, temp_candidates]).reset_index(drop=True)

# Labeling 0 or 1(1 for purchased, 0 for not, but exist in candidates)
total_candidates_df = total_candidates_df\
                          .merge(train, on=['customer_id', 'article_id'], how='left',indicator=True)\
                          .query('_merge == "left_only"').drop('_merge', axis=1)
total_candidates_df['label'] = [0]*total_candidates_df.shape[0]
train_df = train.copy()
train_df['label'] = [1]*train_df.shape[0]
train_df = train_df.sort_values(by = ['customer_id', 'article_id']).reset_index(drop=True)

train_df = pd.concat([train_df, total_candidates_df]).reset_index(drop=True)
train_df = train_df.sort_values(by=['customer_id', 'article_id']).reset_index(drop=True)
train_df['label'] = train_df['label'].astype(int)
train_group = train_df.groupby(['customer_id'])['article_id'].count().values

- 위 코드는 추천된 후보군들을 Ranking 모델에서 사용하기 적합하게 데이터를 전처리해주는 작업이다.

    > - train 데이터에 존재하는 모든 ```customer_id```를 중복이 없도록 불러와서 list를 만들어준다.  
    > - 이후 ```customer_id```: ```article_id```후보군 형태로 저장되어 있던 dictionary를 참고하여, 아래 그림과 같이 한줄에 ```cusotmer_id``` ```article_id```가 하나씩 존재하도록 DataFrame을 만들어준다.  
    >     - training 데이터에 있는 고객들만 130만명이 넘기에 최종결과물이 되는 DataFrame은 5억줄이 넘어가 된다. 이때 한줄을 추가할 때마다 최종 결과물이 되는 ```total_candidates_df```를 호출하여 작업을 진행한다면, 해당 변수를 불러들이고 수정하는 데에 시간이 점차 많이 들게되어 병목현상이 발생하게 된다. 이를 방지하기 위해 10,000단위씩 끊어서 ```temp_candidates```에 저장했다가 ```total_candidates_df```에 이어붙이는 방식으로 데이터 가공을 진행하며, 해당 과정을 진행하기 위해 똑같은 코드 문단을 반복하게 된다.  
    > <table>
        <tr>
        <td><img src="./images/after_processing.png"><figcaption class="figure-caption text-center">Data Frame 변환 이후</figcaption></td>
        </tr>
        </table>  
    > - 마지막엔 해당 고객이 실제 구매를 했으면 1로 label을, 후보군으로 추천은 됐는데 실제 구매기록이 없다면 0으로 label을 하고 저장한다. ```train_group```은 각 고객별로 추천도니 후보군에 article이 몇 개씩 포함되는지를 나타내는데, 이는 Ranking 모델에서 학습을 진행하는데 필요한 변수이다.

In [ ]:
# all customer id's in validation data
validation_customer_list = validation_df['customer_id'].unique()

# Turn candidates dictionary into DataFrame
temp_id = validation_customer_list[0]
df = {'customer_id': [temp_id]*len(customer_candidates_dict[temp_id]),\
      'article_id': customer_candidates_dict[temp_id]}
temp_df = pd.DataFrame(data = df)
val_candidates = temp_df.copy()

for temp_id in tqdm(validation_customer_list[1:]):
    df = {'customer_id': [temp_id]*len(customer_candidates_dict[temp_id]),\
          'article_id': customer_candidates_dict[temp_id]}
    val_candidates = pd.concat([val_candidates, pd.DataFrame(data = df)]).reset_index(drop=True)

# Labeling 0 or 1(1 for purchased, 0 for not, but exist in candidates)
val_candidates = val_candidates\
                     .merge(validation_df, on=['customer_id', 'article_id'], how='left', indicator=True)\
                     .query('_merge == "left_only"').drop('_merge', axis=1)
val_candidates['label'] = [0]*val_candidates.shape[0]
validation_df['label'] = [1]*validation_df.shape[0]
validation_df = pd.concat([validation_df, val_candidates]).reset_index(drop=True)
validation_df = validation_df.sort_values(by=['customer_id', 'article_id']).reset_index(drop=True)
validation_group = validation_df.groupby(['customer_id'])['article_id'].count().values

- validation 데이터에 대해서도 train 데이터와 같이 DataFrame으로 바꿔주고 labeling을 진행한다.

### Customer & Article 메타 데이터 추가

In [5]:
# customer, article meta data attach!
articles = pd.read_csv('articles.csv')
articles_df = articles.drop(columns = ['product_code', 'prod_name', 'detail_desc', 'index_group_name',\
                                       'perceived_colour_value_name', 'index_name', 'perceived_colour_master_name',\
                                       'garment_group_name', 'graphical_appearance_name', 'colour_group_name',\
                                       'section_name', 'product_type_name', 'department_name'])
articles_factorized_df = pd.DataFrame(articles_df['article_id'])
for column in articles_df.columns[1:]:
     articles_factorized_df = pd.concat([articles_factorized_df, pd.DataFrame(articles_df[column].factorize()[0],\
                                                                              columns=[column])], axis=1)

customers = pd.read_csv('customers.csv')
customers_df = customers.drop(columns=['postal_code'])
customers_factorized_df = pd.DataFrame(customers_df['customer_id'])
for column in customers_df.columns[1:]:
    customers_factorized_df = pd.concat([customers_factorized_df, pd.DataFrame(customers_df[column].factorize()[0],\
                                                                               columns=[column])], axis=1)

# (Left) Outer Join
train_df = train_df.merge(articles_factorized_df, how='left', left_on = 'article_id', right_on = 'article_id')
train_df = train_df.merge(customers_factorized_df, how='left', left_on = 'customer_id', right_on = 'customer_id')
validation_df = validation_df.merge(articles_factorized_df, how='left', \
                                    left_on = 'article_id', right_on = 'article_id')
validation_df = validation_df.merge(customers_factorized_df, how='left', \
                                    left_on = 'customer_id', right_on = 'customer_id')

# train, validation data save
pickle.dump([train_df, train_group], open("./train_materials.npy", 'wb'), protocol=4)
np.save('./validation_materials.npy', [validation_df, validation_group])

- 결국 Ranking 모델이 하는 역할은 customer가 article을 구매할 지 아닐지에 대한 확률을 계산하는 것이다. a라는 고객에게 수백개의 article 후보군을 주고 그 중 가장 구매확률이 높은 12개를 추천해주는 것인데, 고객과 article의 고유 id 자체에는 어떠한 정보도 담겨있지 않기 때문에 각 고객과 article의 특징을 DataFrame에 추가해주어야 한다.
- Article은 ```articles.csv```에 저장된 메타데이터를 추가해 줄 것이다. 다만, ```product_code```, ```product_code_name```처럼 불필요하게 중복되는 경우엔 name을 drop해주었다.
- Customer는 ```custoemrs.csv```에 저장된 메타데이터를 추가해 줄 예정인데, article과 같이 중복되는 column은 없었지만, ```1.EDA```에서 언급한 것처럼 고객의 주소를 나타내는 ```postal_code```가 암호화처리되어 유용하게 사용하긴 힘들기에 drop해주었다.
- ***사용할 column들을 선별한 후, pandas의 categorical variable들을 integer로 변환해주는 ```factorize()```라는 함수를 적용한 후, ```customer_id```, ```article_id```, ```label```만 있던 DataFrame에 위에서 언급한 메타데이터들을 Outer Join으로 결합시켜주었다. Ranking 모델은 이렇게 추가된 Customer와 Article의 정보를 인식하고, label을 통해 supervised learning을 진행할 것이다.***
<table>
<tr>
<td><img src="./images/before_join.png"><figcaption class="figure-caption text-center">Before join</figcaption></td>
</tr>
<tr>
<td><img src="./images/after_join.png"><figcaption class="figure-caption text-center">After join</figcaption></td>
</tr>
</table>
- 작업이 끝난 후 변수를 저장하는데, train데이터는 워낙 크기가 커서 ```np.save()```로 저장을 할 수 없어, 다른 protocol을 사용하는 ```pickle.dump()``` 함수를 통해 저장하였다.

## 1. LightGBM Ranker 학습
    - Train, Validation 데이터 load
    - LightGBM Ranker training

- Training 데이터로 130만이 넘는 고객들이 주어졌기에 데이터는 상당히 방대하다. 실제 상기의 전처리 과정을 통해 생성된 training을 위한 DataFrame은 5억줄이 넘는다. 따라서 빠르면서도 정확도 높은 추천을 하기 위해 LightGBM모델을 사용할 것이다.(From Microsoft) LightGBM은 Decision Tree를 base learner로 삼아 이를 sequential하게 연결하여 구성된 ensemble 모델이다. LightGBM은 tabular 데이터에 대해 별다른 전처리에 대한 고민없이 사용할 수 있어 사용이 용이하다. 모델에 대한 자세한 설명은 아래에서 parameter 설명과 더불어 자세히 할 것이다.

### Train, Validation 데이터 load

In [2]:
#train, validation data load
with open('./train_materials.npy', 'rb') as train_load:
    train_df, train_group = pickle.load(train_load)
validation_df, validation_group = np.load('./validation_materials.npy', allow_pickle = True)

### LightGBM Ranker training

In [2]:
from lightgbm.sklearn import LGBMRanker

In [ ]:
# Declare
ranker_one = LGBMRanker(objective = "lambdarank", metric = "map", boosting_type = "dart",
    importance_type = 'gain', n_jobs = 64,                 
    max_depth = 8, n_estimators = 250)

# train
ranker_one.fit(
    train_df.drop(columns=['customer_id', 'article_id', 'label']),
    train_df['label'],
    group = train_group,
    eval_set = [(validation_df.drop(columns=['customer_id', 'article_id', 'label']), validation_df['label'])],
    eval_group = [validation_group],
    eval_at = [12]
    )

# Save model
joblib.dump(ranker_one, 'lgbranker_one.pkl')
# [164]	valid_0's map@12: 0.0607649

- LightGBM Parameters
- ```Model```: Ranking을 계산하는 것이므로 ```LGBMRanker```를 사용한다.(```LGBMRegressor```나 ```LGBMClassifier```도 존재)
- ```Objective function```: Objective function으로 ```lambdarank```를 사용한다. ```lambdarank```는 ranking을 계산할 때 사용하는데, cost(loss) function과 평가 metric간의 목적성의 차이로 인한 성능 저하를 막고자, cost function의 gradient를 이용해 학습을 진행한다.
    > 예를 들어, 여타의 다른 objective function을 사용했을 때를 가정해보자. Ranking 문제의 간단한 예시로, 100개의 item 중 10개를 추천하여 고객이 원하는 item을 맞춰야 한다고 해보자. 이때 고객이 실제로 3개의 item을 구매했고 이를 통해 validation을 진행한다. 모델의 학습 진행 중 정답에 해당되는 3개의 item이 각각 (1위, 10위, 100위)와 (2위, 11위, 50위)의 결과를 보여주었다면, 전자는 10위 안에 실제 구매 item이 2개나 들도록 정답을 맞췄으나 후자는 10위 안에 1개만 존재하므로, 상위 10개만 고려하는 ranking 평가 metric 기준으로 보면 전자가 당연히 더 우수한 결과물이다. 하지만 후자는 1->2위, 10->11위 등으로 높은 순위의 item들에 대해서는 1단계씩만 밀려나고 낮은 순위의 item에 대해서는 100위->50위가 되었으므로, objective function 기준으로 error를 계산해보면 후자가 더 우수한 결과물이라고 판단하고 후자와 같은 방향으로 학습이 진행될 수도 있다. 위와 같은 상황을 방지하고자 ```lambdarank```에선 cost를 직접적으로 사용하지않고, (모델별로 score를 채점하고 이에 통해 계산한 cost의) gradient를 사용한다. 
- ```metric```: metric으론 ```map```를 사용한다. 해당 competition이 ```MAP@12```로 채점되기 때문이다.
- ```boosting_type```: boosting_type으로는 ```dart```를 사용한다. ```dart```는 regression tree에 dropout을 더한 방법으로, LightGBM 공식 document에서 높은 정확도를 위해서 ```dart```를 권장하기 때문에 이를 사용했다.
- ```importance_type```: LightGBM은 decision tree를 여러 개 조합해서 사용하는 ensemble 방법을 사용한다. decision tree는 가지를 뻗게 되는 split을 할 때, split을 함으로써 얻게 되는 information gain이 최대가 되는 지점에서 split을 한다. importance type을 ```gain```으로 하면, split을 할 때 사용한 feature의, 학습을 진행하며 발생한 gain들의 총합들을 나타낸다. 바로 아래의 cell에서 feature 별로 저장된 gain의 합과 비율을 나타내보았다.(```split```값을 준다면 split을 할 때 사용한 횟수를 feature마다 보여준다.)
    > 아래 cell에서 출력된 내용을 보면 ```department```, ```product_type```등이 split을 할 때 가장 많이 사용되었다는 것을 알 수 있고, 이는 해당 변수들이 유효한 변수였다는 의미이다. 상위 변수들을 살펴보면 모두 article의 feature이고, 하위 변수를 살펴보면 customer의 feature이다. 이를 통해 ranking 모델이 작동할 때 article의 특성을 훨씬 많이 이용한다는 것을 알 수 있었다.(약 99%) 99%라는 숫자를 통해 ranker의 작용을 유추해보자면 고객의 정보를 통해 해당 고객이 살만한 article을 추천해주었다기보단, article들의 속성들을 고려하여 사실상 clustering을 통한 추천을 하고 있을 것이라 유추해볼 수 있다. 이게 제대로 작동되는 건가 싶은 생각도 들겠지만, 고객의 개인화된 정보가 age말고는 사실상 유의미해 보이는 변수들이 부족하기에(심지어는 성별 데이터도 없다) 고객들의 구매 log들을 최대한 반영하여 구매 패턴을 일반화하여 학습이 진행되는 것은 당연한 수순이라고 볼 수 있다.
- ```n_jobs```: 사용할 thread의 수를 지정한다.                 
- ```max_depth```: LightGBM의 base learner인 decision tree에 대한 parameter이다. Decision tree가 split할 때 우선순위로 두는 것이 무엇인지에 따라 2가지 종류로 나뉘는데, BFS(Breadth First Search)같이 확장해가는 Level-wise tree가 있고, DFS(Depth First Search)처럼 확장해가는 Leaf-wise tree가 있다. LightGBM은 후자인 Leaf-wise tree인데, over-fitting을 방지하기 위해 뻗어나가는 depth의 최대 깊이를 잘 설정해주어야 한다. 해당 parameter의 tuning을 위해 여러 값을 시도한 결과 8이 최적의 값임을 알 수 있었다.(split 이후 생기는 node들을 나무의 잎사귀와 같이 leaf라고 부르는데, tree당 최대 leaf 수를 제한하는 ```num_leaves```도 leaf-wise tree에서 overfitting을 방지하는데 중요한 요소이다. 이 역시 여러 값을 시도한 결과 default로 설정된 31이 최적의 값임을 확인할 수 있었다.)
- ```n_estimators```: 위에서 LightGBM은 decision tree를 조합하여 모델을 훈련시킨다고 언급했다. 조합 방법에는 여러가지가 있지만 LightGBM은 sequential하게 쌓아나가는데, 앞 tree의 결과로 나온 error를 뒤에 연속되는 tree에서 계속 보정해나가는 방법으로 훈련이 진행된다. 이 tree를 몇개나 쌓을 것인지를 나타내는 것이 ```n_estimators```이다. 
- 기타: ```model.fit()```을 통해 훈련이 진행되는데, 각 tree를 생성할 때마다 validaion을 진행할 수 있다. Competetion에서 원한 것은 ```MAP@12```이므로 그에 맞게 변수를 지정해주었다.

In [30]:
feature_importances_ratio = \
    np.ndarray.round(ranker_load.feature_importances_/ ranker_load.feature_importances_.sum()*100, 2)
feature_importance_df = pd.DataFrame([pd.Series(ranker_load.feature_name_),\
                                      pd.Series(ranker_load.feature_importances_),\
                                      pd.Series(feature_importances_ratio)]).transpose()
feature_importance_df.columns = ['feture_name', 'feature_importances', 'feature_importances_ratio(%)']
feature_importance_df.sort_values(by=['feature_importances'], ascending=False)

,feture_name,feature_importances,feature_importances_ratio(%)
6,department_no,6703377.259949,20.82
0,product_type_no,4784573.469604,14.86
2,graphical_appearance_no,3896802.449219,12.1
10,garment_group_no,3487373.226196,10.83
3,colour_group_code,3207776.063721,9.96
5,perceived_colour_master_id,2811050.594116,8.73
9,section_no,2789415.449829,8.66
1,product_group_name,1821272.788025,5.66
4,perceived_colour_value_id,1330272.203369,4.13
7,index_code,767539.089905,2.38


## 2. Ranker를 활용한 추천 및 검증
    - 고객별 추천(with validation data)
    - MAP@12(with validation data)

### 고객별 추천(with validation data)

In [3]:
# Trained model load
ranker_load = joblib.load('lgbranker_one.pkl')

In [ ]:
# key: customer_id, value: list of 12 article_id
article_rec_dict = dict()
start_index = 0
for group_num in tqdm(validation_group):
    temp_df = validation_df.iloc[start_index:start_index+group_num].reset_index(drop=True)
    temp_pred = ranker_load.predict((temp_df.drop(columns=['customer_id', 'article_id',\
                                                           'label'])))
    temp_top_twelve_indices = np.argsort(temp_pred)[-12:][::-1]
    temp_recommended_articles = temp_df.iloc[temp_top_twelve_indices]['article_id']\
                                       .to_numpy()
    article_rec_dict[temp_df.iloc[0]['customer_id']] = temp_recommended_articles
    start_index += group_num

- Training이 완료된 ranker 모델과 validation 데이터에 등장한 고객들에 대한 후보군을 불러와서, 각 고객에 대해 상위 12개의 article들을 추천할 것이다. 각 고객과 그에 따른 후보군을 모델에 입력값으로 넣어주면, return되는 predicted value 중 가장 높은 값부터 차례대로 12개씩 골라 dictionary에 넣어주면 된다. ```article_rec_dict```는 customer_id를 key로, 12개의 article들을 value로 가지는 dictionary이다. 아래는 저장된 key: value의 한 예시이다.

In [7]:
print('customer_id:', temp_df.iloc[0]['customer_id'])
print('recommended articles:', article_rec_dict[temp_df.iloc[0]['customer_id']])

customer_id: fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20e02ce5d1e58a8f700b
recommended articles: [559633020 748269010 844030001 748582001 715343007 811777007 762796013
 762796006 821152004 803757014 741985002 579541026]


### MAP@12(with validation data)

In [7]:
# correct(label == 1) data extraction
val_correct_np = validation_df[validation_df['label']==1][['customer_id', 'article_id']]\
                     .groupby('customer_id')['article_id'].apply(np.array).reset_index()\
                     .to_numpy()

In [ ]:
average_precision_list = list()
for customer_id, article_array in tqdm(val_correct_np):
    recommended_articles = article_rec_dict[customer_id]
    
    num_hits = 0
    score = 0
    for i, recommended_article in enumerate(recommended_articles):
        if recommended_article in article_array:
            num_hits += 1
            score += num_hits / (i+1)
            
    average_precision = score / min(len(article_array), 12)
    average_precision_list.append(average_precision)
    
mean_average_precision_at_twelve = np.array(average_precision_list).mean()

In [28]:
mean_average_precision_at_twelve

0.06015004603423834

- ```val_correct_np```에서 validation 데이터에 존재하는 고객에 대해, 각 고객별로 실제 구매한 article들을 numpy 형태로 추출했다.
- 그 다음 cell에선 이를 바로 위에서 12개씩의 article을 추천한 결과와 비교하여 MAP@12를 구한 것이다. 결과로 0.06015 정도가 나왔다.

### 성능 평가에 대한 고찰

- 이전에 언급했던 것처럼 해당 Kaggle competition의 최종 우승자가 제출한 모델의 test 데이터에 대한 MAP@12가 0.037 정도였다. 하지만 validation 데이터에 적용한 필자의 모델은 MAP@12가 0.060정도이다. 물론 Test 데이터와 Validation 데이터는 분명 차이가 있을 것이기에 단순히 필자의 모델이 competition 우승자 모델보다 0.060/0.037 = 1.62배 좋다고는 말할 수 없다. 
- ***하지만 valdation 데이터를 구성할 때 Data leakage가 일어나지 않도록 매우 유의하며 작업을 진행했고, validation 데이터를 최대한 test 데이터와 비슷하도록 설정해주었기에(같은 1주일 기간, 1주일밖에 차이나지 않는 시간대 등), test 데이터에 적용해도 매우 커다란 성능감소는 일어나지 않을 것이다. 그러므로 candidate 모델에 이어 ranking 모델까지 적용한 결과는 해당 competition의 우승자 solution과 비교해봐도 유의미한 성능이 나왔다고 생각한다.***